In [1]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import pandas as pd
import csv


# Import matplotlib libraries
# from matplotlib import pyplot as plt
# from matplotlib.collections import LineCollection
# import matplotlib.patches as patches

# import imageio
# from IPython.display import HTML, display


In [2]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [3]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[6:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/train/*")
    save_dir = "dataset/train"
    

    for path in video_paths:
        save_frame(path, save_dir, gap=30)
    
    video_paths = glob("Yoga_Vid_Collected/test/*")
    save_dir = "dataset/test"
    
    for path in video_paths:
        save_frame(path, save_dir, gap=30)

train\abhay_bhuj
abhay_bhuj
dataset/train\Bhujangasana\abhay_bhuj
train\abhay_padam
abhay_padam
dataset/train\Padamasana\abhay_padam
train\abhay_shav
abhay_shav
dataset/train\Shavasana\abhay_shav
train\abhay_tada
abhay_tada
dataset/train\Tadasana\abhay_tada
train\abhay_trik
abhay_trik
dataset/train\Trikonasana\abhay_trik
train\abhay_vriksh
abhay_vriksh
dataset/train\Vrikshasana\abhay_vriksh
train\ameya_bhuj
ameya_bhuj
dataset/train\Bhujangasana\ameya_bhuj
train\ameya_padam
ameya_padam
dataset/train\Padamasana\ameya_padam
train\ameya_shav
ameya_shav
dataset/train\Shavasana\ameya_shav
train\ameya_tadasan
ameya_tadasan
dataset/train\Tadasana\ameya_tadasan
train\ameya_trikonasana
ameya_trikonasana
dataset/train\Trikonasana\ameya_trikonasana
train\ameya_vriksh
ameya_vriksh
dataset/train\Vrikshasana\ameya_vriksh
train\bhumi_bhuj
bhumi_bhuj
dataset/train\Bhujangasana\bhumi_bhuj
train\bhumi_padam
bhumi_padam
dataset/train\Padamasana\bhumi_padam
train\bhumi_shav
bhumi_shav
dataset/train\Shavasa

In [ ]:
def keypts_abs(keypoints):
  kpts_x = keypoints[0, 0, :, 0]
  kpts_y = keypoints[0, 0, :, 1]
  kpts_scores = keypoints[0, 0, :, 2]

  print(kpts_x.shape)
  kpts_absolute_xy = np.stack(
      [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)

  kpts_absolute_xy.shape

  keypoint_threshold=0.2
  kpts_above_thresh_absolute = kpts_absolute_xy[
      kpts_scores > keypoint_threshold, :]
  return kpts_above_thresh_absolute

In [47]:
data=[]
KEYPOINT_DICT = [
    'nose',
    'left_eye',
    'right_eye',
    'left_ear',
    'right_ear',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle'
]
for p in KEYPOINT_DICT:
        data.append(p + "_x")
        data.append(p + "_y")
        data.append(p + "_vis")
data.append( "_label")
data = pd.DataFrame(columns = data) # Empty dataset
#print(data)
data.to_csv("testfile.csv") # save the data as a csv file
# Load the input image.
def image_to_movenet(path,keypts_labels):
    
    if (path.find("Bhujangasana") != -1) :
          label="Bhujangasana"  
    elif (path.find("Padamasana") != -1):
          label="Padamasana"
    elif (path.find("Shavasana") != -1):
          label="Shavasana"
    elif (path.find("Trikonasana") != -1):
          label="Trikonasana"
    elif (path.find("Vrikshasana") != -1):
          label="Vrikshasana"
    elif (path.find("Tadasana") != -1):
          label="Tadasana"
        
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    kpts_x = keypoints[0, 0, :, 0]
    kpts_y = keypoints[0, 0, :, 1]
    kpts_scores = keypoints[0, 0, :, 2]
    
    keypoints_tup=(keypoints,label)
    keypts_labels.append(keypoints_tup)
    print(keypoints)
    
    
#     print(kpts_x)
#     print(kpts_x.numpy())
     # Empty dataset
    t_x=kpts_x.numpy()
    t_y=kpts_y.numpy()
    #t_s=kpts_scores.numpy()
    tn=[]
    for i in range(0,17):
        tn.append(t_x[i])
        tn.append(t_y[i])
    tn.append(label)
     #convert to Numpy array
    df = pd.DataFrame(columns=tn) #convert to a dataframe
    df.to_csv("testfile.csv")
    
#     with open('dataset3.csv','w') as out:
#         csv_out=csv.writer(out)
#        csv_out.writerow(['nose_x', 'nose_y', 'nose_vis', 'left_eye_x', 'left_eye_y', 'left_eye_vis', 'right_eye_x', 'right_eye_y', 'right_eye_vis', 'left_ear_x', 'left_ear_y', 'left_ear_vis', 'right_ear_x', 'right_ear_y', 'right_ear_vis', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_vis', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_vis', 'left_elbow_x', 'left_elbow_y', 'left_elbow_vis', 'right_elbow_x', 'right_elbow_y', 'right_elbow_vis', 'left_wrist_x', 'left_wrist_y', 'left_wrist_vis', 'right_wrist_x', 'right_wrist_y', 'right_wrist_vis', 'left_hip_x', 'left_hip_y', 'left_hip_vis', 'right_hip_x', 'right_hip_y', 'right_hip_vis', 'left_knee_x', 'left_knee_y', 'left_knee_vis', 'right_knee_x', 'right_knee_y', 'right_knee_vis', 'left_ankle_x', 'left_ankle_y', 'left_ankle_vis', 'right_ankle_x', 'right_ankle_y', 'right_ankle_vis', 'label'])
#         for i in range(0,17):
#             csv_out.writerows(kpts_x[i].numpy())
#             csv_out.writerows(kpts_y[i].numpy())
#             csv_out.writerows(kpts_scores[i].numpy())
#         csv_out.writerows(label)
        

#    
#     print(path)
#     print(keypts_labels)
    
    

In [ ]:
import os
import time

path_lst=[]
keypts_labels_train=[]
keypts_labels_test=[]


class_list=os.listdir('dataset/train')
for name in class_list:
    for image in os.listdir(f"dataset/train/{name}"):
        for item in os.listdir(f"dataset/train/{name}/{image}"):
             path_lst.append(os.path.abspath(f"dataset/train/{name}/{image}/{item}"))



path_lst = sorted(path_lst,key=os.path.getmtime)
for path in path_lst:
    image_to_movenet(path,keypts_labels_train)

# class_list=os.listdir('dataset/test')
# for name in class_list:
#     for image in os.listdir(f"dataset/test/{name}"):
#         for item in os.listdir(f"dataset/test/{name}/{image}"):
#              path_lst.append(os.path.abspath(f"dataset/test/{name}/{image}/{item}"))



# path_lst = sorted(path_lst,key=os.path.getmtime)
# for path in path_lst:
#     image_to_movenet(path,keypts_labels_test)    


tf.Tensor(
[[[[0.78621894 0.87021947 0.50614953]
   [0.7801042  0.87738633 0.4593199 ]
   [0.77937704 0.87538415 0.50933564]
   [0.7547425  0.86347234 0.5314946 ]
   [0.75545913 0.86047244 0.5078999 ]
   [0.7497095  0.8122004  0.5055896 ]
   [0.7793317  0.81107634 0.69365615]
   [0.7556962  0.6882595  0.255724  ]
   [0.7984366  0.6940263  0.56903315]
   [0.78506917 0.6002404  0.2885837 ]
   [0.8078834  0.58478236 0.49464354]
   [0.73355955 0.57866436 0.6360941 ]
   [0.74406546 0.5759433  0.59036964]
   [0.7438633  0.43917832 0.64149356]
   [0.77175796 0.43098912 0.42472148]
   [0.7200045  0.33020777 0.3187891 ]
   [0.7472891  0.30311382 0.38912213]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.7752273  0.87912154 0.5893797 ]
   [0.7615156  0.88218987 0.6206248 ]
   [0.7638898  0.880481   0.59206384]
   [0.74586654 0.85848904 0.5556056 ]
   [0.7511187  0.8573169  0.65656596]
   [0.74968785 0.8038533  0.57041734]
   [0.7839574  0.803237   0.5592019 ]
   [0.7541457  0.68630564 

tf.Tensor(
[[[[0.5244836  0.7302153  0.6507486 ]
   [0.5105212  0.7255662  0.5918656 ]
   [0.51455796 0.7153694  0.5830225 ]
   [0.5203596  0.6904832  0.21027234]
   [0.529009   0.6772463  0.6636132 ]
   [0.5842067  0.66105604 0.5925236 ]
   [0.6067718  0.68575704 0.39616096]
   [0.71252596 0.6790048  0.35192883]
   [0.72365165 0.69114655 0.4791616 ]
   [0.78010154 0.6787312  0.2080121 ]
   [0.81422764 0.6948887  0.30798936]
   [0.7263406  0.568563   0.40970194]
   [0.7387674  0.5743927  0.51668006]
   [0.73975503 0.43042475 0.412868  ]
   [0.76846474 0.42942888 0.49272308]
   [0.7361138  0.31172466 0.26313204]
   [0.75826854 0.2822634  0.49637282]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.51316535 0.7342632  0.38710845]
   [0.5015684  0.7240874  0.28439575]
   [0.5051923  0.7202534  0.45978945]
   [0.5132449  0.6903031  0.13038903]
   [0.52276444 0.68613607 0.54520845]
   [0.5850929  0.6623581  0.63271916]
   [0.6040131  0.68628997 0.6280352 ]
   [0.7116957  0.6780658  